In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

# Exercises

## Data Acquisition

- These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.

- This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.

### 1. Read the case, department, and source data into their own spark dataframes.

In [2]:
source = (spark.read.csv("source.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )
source.show(3)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
+---------+----------------+
only showing top 3 rows



In [3]:
source.count(), len(source.columns)

(140, 2)

In [4]:
source.rollup("source_id").count().sort("source_id").show()
# source.where(source['source_id']==127332).show() # there weren't any, try again

+---------+-----+
|source_id|count|
+---------+-----+
|     null|  140|
|   100137|    1|
|   103582|    1|
|   106463|    1|
|   119403|    1|
|   119555|    1|
|   119868|    1|
|   120752|    1|
|   124405|    1|
|   132408|    1|
|   135723|    1|
|   136202|    1|
|   136979|    1|
|   137943|    1|
|   138605|    1|
|   138650|    2|
|   138793|    1|
|   138810|    1|
|   139342|    1|
|   139344|    1|
+---------+-----+
only showing top 20 rows



In [5]:
case = spark.read.csv("case.csv", header=True, inferSchema=True)
case.show(3, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 SLA_due_date         | 1/5/18 8:30          
 case_late            | NO                   
 num_days_late        | -2.0126041

In [6]:
case.count(),len(case.columns)

(841704, 14)

In [7]:
dept = spark.read.csv("dept.csv", header=True, inferSchema=True)
dept.show(3)

+---------------+--------------------+----------------------+-------------------+
|  dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+---------------+--------------------+----------------------+-------------------+
|311 Call Center|    Customer Service|      Customer Service|                YES|
|          Brush|Solid Waste Manag...|           Solid Waste|                YES|
|Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
+---------------+--------------------+----------------------+-------------------+
only showing top 3 rows



### 2. Let's see how writing to the local disk works in spark:

    - Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
    - Inspect your folder structure. What do you notice?

In [8]:
source.write.json("sources_json.json", mode="overwrite")
source.write.csv("sources_csv.csv", mode='overwrite')

# Ravinder's solutions: anything called from 'df'

In [9]:
df = spark.read.csv('case.csv', header=True, inferSchema=True)
print('nrows:', df.count())
# stray_animal_cases = df.filter(df.service_request_type == 'Stray Animal').count()
# print('stray animal cases:', stray_animal_cases)

nrows: 841704


In [10]:
# Rename column
df = df.withColumnRenamed('SLA_due_date', 'case_due_date')

# Convert to better data types
df = (
    df.withColumn('case_late', col('case_late') == 'YES')
    .withColumn('case_closed', col('case_closed') == 'YES')
)
df = df.withColumn('council_district', format_string('%04d', col('council_district')))
df = (
    df.withColumn('case_opened_date', to_timestamp(col('case_opened_date'), 'M/d/yy H:mm'))
    .withColumn('case_closed_date', to_timestamp(col('case_closed_date'), 'M/d/yy H:mm'))
    .withColumn('case_due_date', to_timestamp(col('case_due_date'), 'M/d/yy H:mm'))
)

# Cleanup text data
df = df.withColumn('request_address', lower(trim(col('request_address'))))
# Extract zipcode
df = df.withColumn('zipcode', regexp_extract(col('request_address'), r'\d+$', 0))

# Create a `case_lifetime` feature
df = (
    df.withColumn('case_age', datediff(current_timestamp(), 'case_opened_date'))
    .withColumn('days_to_closed', datediff('case_closed_date', 'case_opened_date'))
    .withColumn('case_lifetime', when(col('case_closed'), col('days_to_closed')).otherwise(col('case_age')))
    .drop('case_age', 'days_to_closed')
)

# Join departments and sources
depts = spark.read.csv('dept.csv', header=True, inferSchema=True)
sources = spark.read.csv('source.csv', header=True, inferSchema=True)

df = df.join(depts, 'dept_division', 'left').join(sources, 'source_id', 'left')

# # Train Test Split
# train, test = df.randomSplit([.8, .2], seed=123)
# train, validate, test = df.randomSplit([.7, .15, .15], seed=123)

In [11]:
df.count()

855269

### 3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [12]:
# source.dtypes # looks good here
# dept.dtypes # the only change I would consider is to make the 'dept_subject_to_SLA' column a boolean
dept = dept.withColumn("dept_subject_to_SLA", expr('dept_subject_to_SLA == "YES"'))
dept.select("dept_subject_to_SLA").show(5)
# looking good

+-------------------+
|dept_subject_to_SLA|
+-------------------+
|               true|
|               true|
|               true|
|               true|
|               true|
+-------------------+
only showing top 5 rows



In [13]:
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [14]:
case.show(1,vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



In [15]:
case = case.withColumnRenamed('SLA_due_date', 'case_due_date')

In [16]:
# use .withColumn to change columns from string to boolean values
case = case.withColumn("case_closed", expr('case_closed == "YES"'))\
.withColumn("case_late", expr('case_late == "YES"'))

In [17]:
case = case.withColumn("council_district", col("council_district").cast("string"))

# my only comment here is if it wouldn't be better to give the districts more 'categorical sounding' names
# eg 'one','two','three', etc


In [18]:
# to_timestamp, fmt

fmt = "M/d/yy H:mm"

case = (
    case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_opened_date", fmt))
)

print("--- After")
case.select("case_opened_date", "case_closed_date", "case_due_date").show(5)

--- After
+-------------------+-------------------+-------------------+
|   case_opened_date|   case_closed_date|      case_due_date|
+-------------------+-------------------+-------------------+
|2018-01-01 00:42:00|2018-01-01 00:42:00|2018-01-01 00:42:00|
|2018-01-01 00:46:00|2018-01-01 00:46:00|2018-01-01 00:46:00|
|2018-01-01 00:48:00|2018-01-01 00:48:00|2018-01-01 00:48:00|
|2018-01-01 01:29:00|2018-01-01 01:29:00|2018-01-01 01:29:00|
|2018-01-01 01:34:00|2018-01-01 01:34:00|2018-01-01 01:34:00|
+-------------------+-------------------+-------------------+
only showing top 5 rows



In [19]:
# request_address: trim and lowercase

case = case.withColumn("request_address", lower(trim("request_address")))

In [20]:
# convert the number of days a case is late to a number of weeks

case = case.withColumn("num_weeks_late", expr("num_days_late / 7"))

In [21]:
case = case.withColumn('zipcode',regexp_extract('request_address', r'(\d+$)',1))

In [22]:
#create three new columns 'case_age', 'days_to_closed', 'case_lifetime'

case = (
    case.withColumn(
        "case_age", datediff(current_timestamp(), "case_opened_date")
    )
    .withColumn(
        "days_to_closed", datediff("case_closed_date", "case_opened_date")
    )
    .withColumn(
        "case_lifetime",
        when(expr("! case_closed"), col("case_age")).otherwise(   ### "when NOT case_closed, then case_age, otherwise...."
            col("days_to_closed")
        ),
    )
)

In [23]:
case.show(2, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 case_due_date        | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 1545                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
-RECORD 1------------------------------------
 case_id              | 1014127333

### How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [24]:
(
case.filter(case['case_closed']=='false')
# .select(col('case_age'))
.withColumn('days_past_SLA', datediff(current_timestamp(),"case_due_date"))
# .show(1,vertical=True)
).select('days_past_SLA').show(1)

# toggling between the commented out code and the .select at the end for different views

+-------------+
|days_past_SLA|
+-------------+
|         1544|
+-------------+
only showing top 1 row



- WRONG ANSWER, APPARENTLY
- Might be bc of the way I prepared my data

In [25]:
df.createOrReplaceTempView('df')

In [26]:
####################################################################
###### Ravinder's answer
#####################################################################

spark.sql('''
SELECT DATEDIFF(current_timestamp, case_due_date) AS days_past_due
FROM df
WHERE NOT case_closed
ORDER BY days_past_due DESC
LIMIT 15
''').show()

+-------------+
|days_past_due|
+-------------+
|         1894|
|         1894|
|         1894|
|         1893|
|         1891|
|         1887|
|         1887|
|         1886|
|         1885|
|         1885|
|         1881|
|         1880|
|         1880|
|         1880|
|         1877|
+-------------+



In [27]:
(
case.select(datediff(current_timestamp(),"case_due_date"))
.where(case['case_closed']=='false')
.show(15)
)
#mmm still not getting what ravinder said

+--------------------------------------------+
|datediff(current_timestamp(), case_due_date)|
+--------------------------------------------+
|                                        1544|
|                                        1544|
|                                        1544|
|                                        1544|
|                                        1544|
|                                        1544|
|                                        1544|
|                                        1543|
|                                        1543|
|                                        1543|
|                                        1543|
|                                        1543|
|                                        1543|
|                                        1543|
|                                        1543|
+--------------------------------------------+
only showing top 15 rows



In [28]:
(
case.filter(case['case_closed']=='false')
# .select(col('case_age'))
.select('case_lifetime').tail(3)
)
# also wrong? our databases are different.  Ravinder says he has seen these answers before : )

[Row(case_lifetime=1547), Row(case_lifetime=1546), Row(case_lifetime=1546)]

In [29]:
# How long has the oldest (in terms of days since opened) currently opened issue been open?
spark.sql('''
SELECT DATEDIFF(current_timestamp, case_opened_date) AS days_past_opened
FROM df
WHERE NOT case_closed
ORDER BY days_past_opened DESC
LIMIT 15
''').show()

+----------------+
|days_past_opened|
+----------------+
|            1910|
|            1910|
|            1909|
|            1908|
|            1907|
|            1907|
|            1907|
|            1907|
|            1907|
|            1907|
|            1907|
|            1907|
|            1907|
|            1907|
|            1907|
+----------------+



### How many Stray Animal cases are there?

In [30]:
case.filter(case['service_request_type']=='Stray Animal').count()

26760

In [31]:
# How many Stray Animal cases are there?
df.filter(df.service_request_type == 'Stray Animal').count()

27361

In [32]:
# another visualization
(
    df.groupBy('service_request_type')
    .count()
    .filter(expr('service_request_type == "Stray Animal"'))
    .show()
)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|        Stray Animal|27361|
+--------------------+-----+



### How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [33]:
# case.show(1, vertical=True)

In [34]:
(
case.where(col('dept_division')=='Field Operations')
.filter(col('service_request_type')!='Officer Standby').count()
# .show(1, vertical=True)
)

113902

In [35]:
# How many service requests that are assigned to the Field Operations department (dept_division)
# are not classified as "Officer Standby" request type (service_request_type)?
(
    df.filter(df.dept_division == 'Field Operations')
    .filter(df.service_request_type != 'Officer Standby')
    .count()
)

116295

In [36]:
# Another way to do it
(
    df.filter(expr("dept_division == 'Field Operations'"))
    .filter(expr('service_request_type != "Officer Standby"'))
    .count()
)

116295

### Convert the council_district column to a string column.

In [37]:
# I did this in an above step

### Extract the year from the case_closed_date column.

In [38]:
# case.show(1, vertical=True)

In [39]:
case = case.withColumn('case_closed_year', year('case_closed_date'))
case.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 case_due_date        | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 1545                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 case_closed_year     | 2018                 
only showing top 1 row



### Convert num_days_late from days to hours in new columns num_hours_late.

In [40]:
# case.show(1, vertical=True)

In [41]:
case = case.withColumn('num_hours_late', col('num_days_late') * 24)
case.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 case_due_date        | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 1545                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 case_closed_year     | 2018                 
 num_hours_late       | -23964.210

### Join the case data with the source and department data.

In [42]:
dept.show(2)

+---------------+--------------------+----------------------+-------------------+
|  dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+---------------+--------------------+----------------------+-------------------+
|311 Call Center|    Customer Service|      Customer Service|               true|
|          Brush|Solid Waste Manag...|           Solid Waste|               true|
+---------------+--------------------+----------------------+-------------------+
only showing top 2 rows



In [43]:
source.show(2)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
+---------+----------------+
only showing top 2 rows



In [44]:
source_counts = source.groupBy("source_id").count().sort("count")
source_counts.show(5)

+---------+-----+
|source_id|count|
+---------+-----+
|   136202|    1|
|   141239|    1|
|  MW16328|    1|
|  df03076|    1|
|   141549|    1|
+---------+-----+
only showing top 5 rows



In [45]:
source_counts.count()

139

In [46]:
case_counts = case.groupBy("source_id").count().sort("count")
case_counts.count()

139

In [47]:
case.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 case_due_date        | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 1545                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 case_closed_year     | 2018                 
 num_hours_late       | -23964.210

In [48]:
# join the df and dept dataframe using 'dept_division' as common key
# drop columns as needed (keep standardized_dept_name)
# convert dept_subject_to_SLA to boolean

case = (
    (
    case
    # left join on dept_division
    .join(dept, "dept_division", "left")
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(dept.dept_division)
    .drop(dept.dept_name)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
    )
)

In [49]:
case = case.join(source, case.source_id == source.source_id, 'outer').drop(case.source_id)

In [50]:
case.show(1,vertical=True)

-RECORD 0------------------------------------
 dept_division        | Waste Collection     
 case_id              | 1014128056           
 case_opened_date     | 2018-01-02 08:21:00  
 case_closed_date     | 2018-01-02 08:21:00  
 case_due_date        | 2018-01-02 08:21:00  
 case_late            | true                 
 num_days_late        | 5.00681713           
 case_closed          | true                 
 service_request_type | Solid Waste Fees ... 
 SLA_days             | 3.00619213           
 case_status          | Closed               
 request_address      | 3214  stoney fork... 
 council_district     | 10                   
 num_weeks_late       | 0.71525959           
 zipcode              | 78247                
 case_age             | 1544                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 case_closed_year     | 2018                 
 num_hours_late       | 120.16361112         
 department           | Solid Wast

### Are there any cases that do not have a request source?

In [51]:
from pyspark.sql.functions import isnan, when, count, col


df_columns = ['source_id','source_username']
case.select([count(when(isnan(c)| col(c).isNull(), c)).alias(c) for c in df_columns]).show()



# i think this is correct, if i have understood the question 

+---------+---------------+
|source_id|source_username|
+---------+---------------+
|        0|              0|
+---------+---------------+



In [52]:
# ravinder's solution, confirming mine:
df.filter(col('source_id').isNull()).show(vertical=True)

(0 rows)



### What are the top 10 service request types in terms of number of requests?

In [53]:
(
case
.groupBy('service_request_type')
.agg(count(col('service_request_type')))
.sort(count(('service_request_type')).desc())
.show(10, vertical=True)
)

-RECORD 0-------------------------------------------
 service_request_type        | No Pickup            
 count(service_request_type) | 89210                
-RECORD 1-------------------------------------------
 service_request_type        | Overgrown Yard/Trash 
 count(service_request_type) | 66403                
-RECORD 2-------------------------------------------
 service_request_type        | Bandit Signs         
 count(service_request_type) | 32968                
-RECORD 3-------------------------------------------
 service_request_type        | Damaged Cart         
 count(service_request_type) | 31163                
-RECORD 4-------------------------------------------
 service_request_type        | Front Or Side Yar... 
 count(service_request_type) | 28920                
-RECORD 5-------------------------------------------
 service_request_type        | Stray Animal         
 count(service_request_type) | 27361                
-RECORD 6-------------------------------------

In [54]:
# What are the top 10 service request types in terms of number of requests?
(
    df.groupby('service_request_type')
    .count()
    .sort(col('count').desc())
    .show(10, truncate=False)
)

# good job on your answer, this is just better visually

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



### What are the top 10 service request types in terms of average days late?

In [55]:
(
case
.groupBy('service_request_type')
.agg(mean(col('num_days_late')))
.sort(mean(('num_days_late')).desc())
.show(10, vertical=True)
)

-RECORD 0------------------------------------
 service_request_type | Zoning: Junk Yards   
 avg(num_days_late)   | 175.95636210420932   
-RECORD 1------------------------------------
 service_request_type | Labeling for Used... 
 avg(num_days_late)   | 162.43032902285717   
-RECORD 2------------------------------------
 service_request_type | Record Keeping of... 
 avg(num_days_late)   | 153.99724039428568   
-RECORD 3------------------------------------
 service_request_type | Signage Requied f... 
 avg(num_days_late)   | 151.63868055333333   
-RECORD 4------------------------------------
 service_request_type | Storage of Used M... 
 avg(num_days_late)   | 142.11255641500003   
-RECORD 5------------------------------------
 service_request_type | Zoning: Recycle Yard 
 avg(num_days_late)   | 135.9285161247979    
-RECORD 6------------------------------------
 service_request_type | Donation Containe... 
 avg(num_days_late)   | 131.75610506358709   
-RECORD 7-------------------------

In [56]:
# What are the top 10 service request types in terms of average days late?
# - just the late cases
# - for the late cases:
#   - what is the average number of days late by request type?
(
    df.where('case_late') # just the rows where case_late == true
    .groupBy('service_request_type')
    .agg(mean('num_days_late').alias('n_days_late'), count('*').alias('n_cases'))
    .sort(desc('n_days_late'))
    .show(10, truncate=False)
)

+--------------------------------------+------------------+-------+
|service_request_type                  |n_days_late       |n_cases|
+--------------------------------------+------------------+-------+
|Zoning: Recycle Yard                  |210.89201994318182|132    |
|Zoning: Junk Yards                    |200.20517608494276|262    |
|Structure/Housing Maintenance         |190.20707698509807|51     |
|Donation Container Enforcement        |171.09115313942615|122    |
|Storage of Used Mattress              |163.96812829714287|7      |
|Labeling for Used Mattress            |162.43032902285717|7      |
|Record Keeping of Used Mattresses     |153.99724039428568|7      |
|Signage Requied for Sale of Used Mattr|151.63868055333333|12     |
|Traffic Signal Graffiti               |137.64583330000002|4      |
|License Requied Used Mattress Sales   |128.79828704142858|7      |
+--------------------------------------+------------------+-------+
only showing top 10 rows



### Does number of days late depend on department?

In [57]:
(
case
.groupBy('department')
.agg(mean(col('num_days_late')))
.sort(mean(('num_days_late')).desc())
.show(vertical=True)
)

-RECORD 0----------------------------------
 department         | Customer Service     
 avg(num_days_late) | 59.73709149630082    
-RECORD 1----------------------------------
 department         | Solid Waste          
 avg(num_days_late) | -2.200057513672168   
-RECORD 2----------------------------------
 department         | Metro Health         
 avg(num_days_late) | -4.911766979607001   
-RECORD 3----------------------------------
 department         | Parks & Recreation   
 avg(num_days_late) | -5.251521960055156   
-RECORD 4----------------------------------
 department         | Trans & Cap Impro... 
 avg(num_days_late) | -20.612837354052726  
-RECORD 5----------------------------------
 department         | DSD/Code Enforcement 
 avg(num_days_late) | -38.3693889261437    
-RECORD 6----------------------------------
 department         | Animal Care Services 
 avg(num_days_late) | -226.5178394055035   
-RECORD 7----------------------------------
 department         | City Counc

In [58]:
# Does number of days late depend on department?
(
    df.filter('case_late')
    .groupby('dept_name')
    .agg(mean('num_days_late').alias('days_late'), count('num_days_late').alias('n_cases_late'))
    .sort('days_late')
    .withColumn('days_late', round(col('days_late'), 1))
    .show(truncate=False)
)

+-------------------------+---------+------------+
|dept_name                |days_late|n_cases_late|
+-------------------------+---------+------------+
|Metro Health             |6.5      |854         |
|Solid Waste Management   |7.1      |33729       |
|Trans & Cap Improvements |10.7     |5529        |
|Parks and Recreation     |22.4     |3810        |
|Animal Care Services     |23.4     |23751       |
|Code Enforcement Services|48.1     |25467       |
|Development Services     |67.2     |840         |
|Customer Service         |88.2     |2035        |
|null                     |210.9    |132         |
+-------------------------+---------+------------+



### How do number of days late depend on department and request type?

In [59]:
# case.show(1,vertical=True)

In [60]:
####################################################################
###### This code works great, don't lose it, do build on it
###### You could add code to filter only those cases that are closed for a closer comparison to ravinder
#####################################################################

(
case
.groupBy('department','service_request_type')
.agg(mean(col('num_days_late')))
.sort(mean(('num_days_late')).desc())
.show(10, vertical=True)
)

-RECORD 0------------------------------------
 department           | DSD/Code Enforcement 
 service_request_type | Zoning: Junk Yards   
 avg(num_days_late)   | 175.95636210420932   
-RECORD 1------------------------------------
 department           | DSD/Code Enforcement 
 service_request_type | Labeling for Used... 
 avg(num_days_late)   | 162.43032902285717   
-RECORD 2------------------------------------
 department           | DSD/Code Enforcement 
 service_request_type | Record Keeping of... 
 avg(num_days_late)   | 153.99724039428568   
-RECORD 3------------------------------------
 department           | DSD/Code Enforcement 
 service_request_type | Signage Requied f... 
 avg(num_days_late)   | 151.63868055333333   
-RECORD 4------------------------------------
 department           | DSD/Code Enforcement 
 service_request_type | Storage of Used M... 
 avg(num_days_late)   | 142.11255641500003   
-RECORD 5------------------------------------
 department           | DSD/Code E

In [61]:
# How do number of days late depend on department and request type?
(
    df.filter("case_closed")
#     .filter("case_late")
    .groupby("standardized_dept_name", "service_request_type")
    .agg(avg("num_days_late").alias("days_late"), count("*").alias("n_cases"))
    .withColumn("days_late", round(col("days_late"), 1))
    .where(col('days_late') > 0)
    .sort(desc("days_late"))
    .show(40, truncate=False)
)



+------------------------+----------------------------------------+---------+-------+
|standardized_dept_name  |service_request_type                    |days_late|n_cases|
+------------------------+----------------------------------------+---------+-------+
|DSD/Code Enforcement    |Zoning: Junk Yards                      |209.4    |174    |
|DSD/Code Enforcement    |Labeling for Used Mattress              |162.4    |7      |
|DSD/Code Enforcement    |Record Keeping of Used Mattresses       |154.0    |7      |
|DSD/Code Enforcement    |Signage Requied for Sale of Used Mattr  |151.6    |12     |
|DSD/Code Enforcement    |Storage of Used Mattress                |142.1    |8      |
|DSD/Code Enforcement    |Donation Container Enforcement          |141.3    |114    |
|DSD/Code Enforcement    |Zoning: Recycle Yard                    |139.0    |141    |
|DSD/Code Enforcement    |License Requied Used Mattress Sales     |128.8    |7      |
|DSD/Code Enforcement    |Vendors                     